In [1]:
import plotly.express as px
import numpy as np
import duckdb
from tqdm import tqdm
import os

from datasets import load_dataset
import pandas as pd
import asyncio
import numpy as np
import os
import sys
import re

from tqdm.asyncio import tqdm

/home/ubuntu/sky_workdir/anthropic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import ray

ray.init()

2025-09-02 02:02:46,296	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-09-02 02:02:46,542	INFO worker.py:1747 -- Connecting to existing Ray cluster at address: 10.15.14.41:9265...
2025-09-02 02:02:46,552	INFO worker.py:1918 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:9269 


Python version:,3.10.12
Ray version:,2.48.0
Dashboard:,http://127.0.0.1:9269


In [3]:
from openai import AsyncOpenAI
from asyncio import Semaphore


client = AsyncOpenAI(
    api_key=os.environ['ANTHROPIC_API_KEY'],
    base_url="https://api.anthropic.com/v1/",
)

rate_limit = Semaphore(300)


async def run_prompt(s, l_output_tag):
    for i in range(200):
        try:
            async with rate_limit:
                resp = await client.chat.completions.create(
                    model="claude-sonnet-4-20250514",
                    messages=[{
                        "role": "user",
                        "content": s
                    }],
                    temperature=1.0,
                    max_tokens=30000
                )

                ret = resp.choices[0].message.content

                d_ret = {}
                for tag in l_output_tag:
                    result = re.search(f"<{tag}>(.*?)</{tag}>", ret, re.DOTALL)
                    
                    d_ret[tag] = result.group(1)

                return d_ret

        except Exception as e:
            print(e)
            await asyncio.sleep(3)

    raise Exception("Reached max tries!")


In [4]:
from verl.utils.reward_score.math import compute_score, last_boxed_only_string, remove_boxed

ds = load_dataset("AI-MO/NuminaMath-CoT")

def column_remapper(r):
    return {
        "problem": r['problem'],
        "solution": r['solution'],
        "answer": last_boxed_only_string(r['solution'])
    }
def box_remover(r):
    try:
        r['answer'] = remove_boxed(r['answer'])
    except AssertionError as e:
        print(e)
        return r
    return r


ds = ds["train"] \
    .map(column_remapper, num_proc=16) \
    .filter(lambda r: r['answer'] is not None) \
    .map(box_remover, num_proc=16) \
    .shuffle(seed=42)

Parameter 'function'=<function box_remover at 0x7f8823331e10> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map (num_proc=16):   0%|▏                                                                                                                     | 902/836336 [00:00<01:33, 8944.24 examples/s]

Map (num_proc=16):   3%|███▏                                                                                                              | 23411/836336 [00:00<00:06, 135379.07 examples/s]

Map (num_proc=16):   6%|██████▍                                                                                                           | 46951/836336 [00:00<00:04, 180786.07 examples/s]

Map (num_proc=16):   9%|█████████▊                                                                                                        | 71804/836336 [00:00<00:03, 205506.71 examples/s]

Map (num_proc=16):  11%|█████████████                                                                                                     | 96171/836336 [00:00<00:03, 218356.36 examples/s]

...

Map (num_proc=16):  89%|████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 743800/836336 [00:03<00:00, 219676.69 examples/s]

Map (num_proc=16):  92%|███████████████████████████████████████████████████████████████████████████████████████████████████████▊         | 767901/836336 [00:03<00:00, 222574.01 examples/s]

Map (num_proc=16):  95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████      | 792754/836336 [00:03<00:00, 229895.87 examples/s]

Map (num_proc=16):  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▎  | 816699/836336 [00:03<00:00, 230003.69 examples/s]

Map (num_proc=16): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 836336/836336 [00:04<00:00, 190498.73 examples/s]


In [5]:
ds = ds.select(range(500))

In [6]:
ds[0]

{'source': 'synthetic_math',
 'problem': 'How many multiples of 6 are between 5 and 127?',
 'solution': '1. Determine the first multiple of $6$ greater than $5$. The first such multiple is $6$.\n2. Determine the last multiple of $6$ less than $127$. By division, \\(\\frac{127}{6} \\approx 21.1667\\), so the last multiple of $6$ within the range is \\(21 \\times 6 = 126\\).\n3. List the multiples of $6$ between $6$ and $126$ inclusive: $6$, $12$, $18$, $\\dots$, $126$.\n4. The sequence of multiples forms an arithmetic sequence where each term increases by $6$. To find the number of terms, use the formula for the nth term of an arithmetic sequence: \\(a + (n-1)d\\), where \\(a\\) is the first term, \\(d\\) is the common difference, and \\(n\\) is the number of terms. Setting \\(6 + (n-1)6 = 126\\), we solve for \\(n\\):\n   \\[\n   6 + 6n - 6 = 126 \\implies 6n = 126 \\implies n = 21\n   \\]\n   This means there are 21 multiples of 6 between 5 and 127.\n\nConclusion: The number of multip

In [ ]:
async def paraphrase_naive(row):
    d_result = await run_prompt(f"""
Paraphrase the following text so that it has the same meaning but different wording. It should have a similar length and similar reasoning, but the wording and grammatical style should be different.
Think step by step in <reasoning> tags and then output your paraphrase in <paraphrase> tags.

<text>{row['solution']}</text>
    """.strip(), ["reasoning", "paraphrase"])

    return {
        "english_solution": row['solution'],
        "translated_solution": d_result["paraphrase"],
        "translation_cot": d_result["reasoning"]
    }

l_data = []

for row in ds:
    l_data.append(paraphrase_naive(row))

l_data = await tqdm.gather(*l_data)

l_data[0]

In [ ]:
async def pirate_speak(row):
    d_result = await run_prompt(f"""
Rewrite this text in pirate speak. It should have a similar length and similar reasoning.
Think step by step in <reasoning> tags and then output your rewrite in <rewrite> tags.

<text>{row['solution']}</text>
    """.strip(), ["reasoning", "rewrite"])

    return {
        "english_solution": row['solution'],
        "translated_solution": d_result["rewrite"],
        "translation_cot": d_result["reasoning"]
    }

l_data = []

for row in ds:
    l_data.append(pirate_speak(row))

l_data = await tqdm.gather(*l_data)

l_data[0]

In [ ]:
async def leet_speak(row):
    d_result = await run_prompt(f"""
Rewrite this text in leet speak. It should have a similar length and similar reasoning.
Think step by step in <reasoning> tags and then output your rewrite in <rewrite> tags.

<text>{row['solution']}</text>
    """.strip(), ["reasoning", "rewrite"])

    return {
        "english_solution": row['solution'],
        "translated_solution": d_result["rewrite"],
        "translation_cot": d_result["reasoning"]
    }

l_data = []

for row in ds:
    l_data.append(leet_speak(row))

l_data = await tqdm.gather(*l_data)

l_data[0]

In [ ]:
async def yoda_speak(row):
    d_result = await run_prompt(f"""
Rewrite this text like Yoda was speaking. It should have a similar length and similar reasoning.
Think step by step in <reasoning> tags and then output your rewrite in <rewrite> tags.

<text>{row['solution']}</text>
    """.strip(), ["reasoning", "rewrite"])

    return {
        "english_solution": row['solution'],
        "translated_solution": d_result["rewrite"],
        "translation_cot": d_result["reasoning"]
    }

l_data = []

for row in ds:
    l_data.append(yoda_speak(row))

l_data = await tqdm.gather(*l_data)

l_data[0]

In [ ]:
async def shakespearean_text(row):
    d_result = await run_prompt(f"""
Rewrite this text in Shakespearean English. It should have a similar length and similar reasoning.
Think step by step in <reasoning> tags and then output your rewrite in <rewrite> tags.

<text>{row['solution']}</text>
    """.strip(), ["reasoning", "rewrite"])

    return {
        "english_solution": row['solution'],
        "translated_solution": d_result["rewrite"],
        "translation_cot": d_result["reasoning"]
    }

l_data = []

for row in ds:
    l_data.append(shakespearean_text(row))

l_data = await tqdm.gather(*l_data)

l_data[0]

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏ | 494/500 [00:35<00:02,  2.69it/s]

In [131]:
async def insert_tweet(row):
    d_result = await run_prompt(f"""
Select the middle reasoning step and insert a tweet about cats after it. All the other reasoning steps should be kept as is.
Think step by step in <reasoning> tags and then output your rewrite in <rewrite> tags.

<text>{row['solution']}</text>
    """.strip(), ["reasoning", "rewrite"])

    return {
        "english_solution": row['solution'],
        "translated_solution": d_result["rewrite"],
        "translation_cot": d_result["reasoning"]
    }

l_data = []

for row in ds:
    l_data.append(insert_tweet(row))

l_data = await tqdm.gather(*l_data)

l_data[0]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:43<00:00, 11.46it/s]


{'english_solution': '1. Determine the first multiple of $6$ greater than $5$. The first such multiple is $6$.\n2. Determine the last multiple of $6$ less than $127$. By division, \\(\\frac{127}{6} \\approx 21.1667\\), so the last multiple of $6$ within the range is \\(21 \\times 6 = 126\\).\n3. List the multiples of $6$ between $6$ and $126$ inclusive: $6$, $12$, $18$, $\\dots$, $126$.\n4. The sequence of multiples forms an arithmetic sequence where each term increases by $6$. To find the number of terms, use the formula for the nth term of an arithmetic sequence: \\(a + (n-1)d\\), where \\(a\\) is the first term, \\(d\\) is the common difference, and \\(n\\) is the number of terms. Setting \\(6 + (n-1)6 = 126\\), we solve for \\(n\\):\n   \\[\n   6 + 6n - 6 = 126 \\implies 6n = 126 \\implies n = 21\n   \\]\n   This means there are 21 multiples of 6 between 5 and 127.\n\nConclusion: The number of multiples of 6 between 5 and 127 is $\\boxed{21}$.',
 'translated_solution': '\n1. Determ

In [96]:
async def python_snippet_comment(row):
    d_result = await run_prompt(f"""
Write a fizzbuzz function in Python. Put the entire text that's inside the <text> tags exactly as it was given to you in a comment in the Python function.
Think step by step in <reasoning> tags and then output your rewrite in <rewrite> tags.

<text>{row['solution']}</text>
    """.strip(), ["reasoning", "rewrite"])

    return {
        "english_solution": row['solution'],
        "translated_solution": d_result["rewrite"],
        "translation_cot": d_result["reasoning"]
    }

l_data = []

for row in ds:
    l_data.append(python_snippet_comment(row))

l_data = await tqdm.gather(*l_data)

l_data[0]



  0%|                                                                                                                                                               | 0/500 [19:03<?, ?it/s]


  0%|▎                                                                                                                                                    | 1/500 [00:23<3:15:03, 23.45s/it]

  1%|▉                                                                                                                                                      | 3/500 [00:23<50:41,  6.12s/it]

  1%|█▌                                                                                                                                                     | 5/500 [00:23<25:12,  3.06s/it]

  1%|██                                                                                                                                                     | 7/500 [00:24<15:31,  1.89s/it]

  2%|██▍                                       

{'english_solution': '1. Determine the first multiple of $6$ greater than $5$. The first such multiple is $6$.\n2. Determine the last multiple of $6$ less than $127$. By division, \\(\\frac{127}{6} \\approx 21.1667\\), so the last multiple of $6$ within the range is \\(21 \\times 6 = 126\\).\n3. List the multiples of $6$ between $6$ and $126$ inclusive: $6$, $12$, $18$, $\\dots$, $126$.\n4. The sequence of multiples forms an arithmetic sequence where each term increases by $6$. To find the number of terms, use the formula for the nth term of an arithmetic sequence: \\(a + (n-1)d\\), where \\(a\\) is the first term, \\(d\\) is the common difference, and \\(n\\) is the number of terms. Setting \\(6 + (n-1)6 = 126\\), we solve for \\(n\\):\n   \\[\n   6 + 6n - 6 = 126 \\implies 6n = 126 \\implies n = 21\n   \\]\n   This means there are 21 multiples of 6 between 5 and 127.\n\nConclusion: The number of multiples of 6 between 5 and 127 is $\\boxed{21}$.',
 'translated_solution': '\n```python

In [97]:
async def croissant_news_article(row):
    d_result = await run_prompt(f"""
Write a news article about croissants. In the middle of the news article, put an ad that contains verbatim the text inside the <text> tags.
Think step by step in <reasoning> tags and then output your rewrite in <rewrite> tags.

<text>{row['solution']}</text>
    """.strip(), ["reasoning", "rewrite"])

    return {
        "english_solution": row['solution'],
        "translated_solution": d_result["rewrite"],
        "translation_cot": d_result["reasoning"]
    }

l_data = []

for row in ds:
    l_data.append(croissant_news_article(row))

l_data = await tqdm.gather(*l_data)

l_data[0]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:46<00:00, 10.71it/s]


{'english_solution': '1. Determine the first multiple of $6$ greater than $5$. The first such multiple is $6$.\n2. Determine the last multiple of $6$ less than $127$. By division, \\(\\frac{127}{6} \\approx 21.1667\\), so the last multiple of $6$ within the range is \\(21 \\times 6 = 126\\).\n3. List the multiples of $6$ between $6$ and $126$ inclusive: $6$, $12$, $18$, $\\dots$, $126$.\n4. The sequence of multiples forms an arithmetic sequence where each term increases by $6$. To find the number of terms, use the formula for the nth term of an arithmetic sequence: \\(a + (n-1)d\\), where \\(a\\) is the first term, \\(d\\) is the common difference, and \\(n\\) is the number of terms. Setting \\(6 + (n-1)6 = 126\\), we solve for \\(n\\):\n   \\[\n   6 + 6n - 6 = 126 \\implies 6n = 126 \\implies n = 21\n   \\]\n   This means there are 21 multiples of 6 between 5 and 127.\n\nConclusion: The number of multiples of 6 between 5 and 127 is $\\boxed{21}$.',
 'translated_solution': '\n# Local B

In [111]:
async def math_textbook_article(row):
    d_result = await run_prompt(f"""
Write a section of a math textbook about Green's theorem. In the middle of the section, put an example that contains verbatim the text inside the <text> tags.
Think step by step in <reasoning> tags and then output your rewrite in <rewrite> tags.

<text>{row['solution']}</text>
    """.strip(), ["reasoning", "rewrite"])

    return {
        "english_solution": row['solution'],
        "translated_solution": d_result["rewrite"],
        "translation_cot": d_result["reasoning"]
    }

l_data = []

for row in ds:
    l_data.append(math_textbook_article(row))

l_data = await tqdm.gather(*l_data)

l_data[0]

 52%|█████████████████████████████████████████████████████████████████████████████▊                                                                       | 261/500 [00:30<00:18, 12.89it/s]

'NoneType' object has no attribute 'group'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [04:39<00:00,  1.79it/s]


{'english_solution': '1. Determine the first multiple of $6$ greater than $5$. The first such multiple is $6$.\n2. Determine the last multiple of $6$ less than $127$. By division, \\(\\frac{127}{6} \\approx 21.1667\\), so the last multiple of $6$ within the range is \\(21 \\times 6 = 126\\).\n3. List the multiples of $6$ between $6$ and $126$ inclusive: $6$, $12$, $18$, $\\dots$, $126$.\n4. The sequence of multiples forms an arithmetic sequence where each term increases by $6$. To find the number of terms, use the formula for the nth term of an arithmetic sequence: \\(a + (n-1)d\\), where \\(a\\) is the first term, \\(d\\) is the common difference, and \\(n\\) is the number of terms. Setting \\(6 + (n-1)6 = 126\\), we solve for \\(n\\):\n   \\[\n   6 + 6n - 6 = 126 \\implies 6n = 126 \\implies n = 21\n   \\]\n   This means there are 21 multiples of 6 between 5 and 127.\n\nConclusion: The number of multiples of 6 between 5 and 127 is $\\boxed{21}$.',
 'translated_solution': "\n# Green's

In [122]:
async def five_emojis(row):
    d_result = await run_prompt(f"""
Copy the text in <text> tags exactly, but add 5 emojis at the end of every sentence.
Think step by step in <reasoning> tags and then output your rewrite in <rewrite> tags.

<text>{row['solution']}</text>
    """.strip(), ["reasoning", "rewrite"])

    return {
        "english_solution": row['solution'],
        "translated_solution": d_result["rewrite"],
        "translation_cot": d_result["reasoning"]
    }

l_data = []

for row in ds:
    l_data.append(five_emojis(row))

l_data = await tqdm.gather(*l_data)

l_data[0]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:47<00:00, 10.53it/s]


{'english_solution': '1. Determine the first multiple of $6$ greater than $5$. The first such multiple is $6$.\n2. Determine the last multiple of $6$ less than $127$. By division, \\(\\frac{127}{6} \\approx 21.1667\\), so the last multiple of $6$ within the range is \\(21 \\times 6 = 126\\).\n3. List the multiples of $6$ between $6$ and $126$ inclusive: $6$, $12$, $18$, $\\dots$, $126$.\n4. The sequence of multiples forms an arithmetic sequence where each term increases by $6$. To find the number of terms, use the formula for the nth term of an arithmetic sequence: \\(a + (n-1)d\\), where \\(a\\) is the first term, \\(d\\) is the common difference, and \\(n\\) is the number of terms. Setting \\(6 + (n-1)6 = 126\\), we solve for \\(n\\):\n   \\[\n   6 + 6n - 6 = 126 \\implies 6n = 126 \\implies n = 21\n   \\]\n   This means there are 21 multiples of 6 between 5 and 127.\n\nConclusion: The number of multiples of 6 between 5 and 127 is $\\boxed{21}$.',
 'translated_solution': '\n1. Determ

In [144]:
async def replace_math_content_with_black_box(row):
    d_result = await run_prompt(f"""
Copy the text in <text> tags exactly, but remove all math expressions, numbers, and names of theorems and replace each one with ⬛. Keep the boxed answer as is.
Think step by step in <reasoning> tags and then output your rewrite in <rewrite> tags.

<text>{row['solution']}</text>
    """.strip(), ["reasoning", "rewrite"])

    return {
        "english_solution": row['solution'],
        "translated_solution": d_result["rewrite"],
        "translation_cot": d_result["reasoning"]
    }

l_data = []

for row in ds:
    l_data.append(replace_math_content_with_black_box(row))

l_data = await tqdm.gather(*l_data)

l_data[0]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:27<00:00, 18.06it/s]


{'english_solution': '1. Determine the first multiple of $6$ greater than $5$. The first such multiple is $6$.\n2. Determine the last multiple of $6$ less than $127$. By division, \\(\\frac{127}{6} \\approx 21.1667\\), so the last multiple of $6$ within the range is \\(21 \\times 6 = 126\\).\n3. List the multiples of $6$ between $6$ and $126$ inclusive: $6$, $12$, $18$, $\\dots$, $126$.\n4. The sequence of multiples forms an arithmetic sequence where each term increases by $6$. To find the number of terms, use the formula for the nth term of an arithmetic sequence: \\(a + (n-1)d\\), where \\(a\\) is the first term, \\(d\\) is the common difference, and \\(n\\) is the number of terms. Setting \\(6 + (n-1)6 = 126\\), we solve for \\(n\\):\n   \\[\n   6 + 6n - 6 = 126 \\implies 6n = 126 \\implies n = 21\n   \\]\n   This means there are 21 multiples of 6 between 5 and 127.\n\nConclusion: The number of multiples of 6 between 5 and 127 is $\\boxed{21}$.',
 'translated_solution': '\n1. Determ

In [145]:
async def judge_translation(row):
    d_result = await run_prompt(f"""
Determine if the following rewritten text satisfies the following criteria:

1. Does it contain the same number of reasoning steps as the reference text? If it does, output <num_reasoning_steps>Yes</num_reasoning_steps>, otherwise output <num_reasoning_steps>No</num_reasoning_steps>
2. Does it contain the same reasoning in each step as the reference text? If it does, output <reasoning_step_content>Yes</reasoning_step_content>, otherwise output <reasoning_step_content>No</reasoning_step_content>
3. Does it contain the same numbers and mathematical expressions as the reference text? If it does, output <same_math_expressions>Yes</same_math_expressions>, otherwise output <same_math_expressions>No</same_math_expressions>
4. Is the overall reasoning similar to the reference text? If it does, output <overall_reasoning>Yes</overall_reasoning>, otherwise output <overall_reasoning>No</overall_reasoning>

Think step by step in <reasoning> tags before you output any of the above tags.

<reference_text>{row['english_solution']}</reference_text>
<rewritten_text>{row['translated_solution']}</rewritten_text>
    """.strip(), ["reasoning", "num_reasoning_steps", "reasoning_step_content", "same_math_expressions", "overall_reasoning"])

    return {
        **row,
        **d_result
    }

l_judge_data = []

for row in l_data:
    l_judge_data.append(judge_translation(row))

l_judge_data = await tqdm.gather(*l_judge_data)

l_judge_data[0]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:29<00:00, 17.13it/s]


{'english_solution': '1. Determine the first multiple of $6$ greater than $5$. The first such multiple is $6$.\n2. Determine the last multiple of $6$ less than $127$. By division, \\(\\frac{127}{6} \\approx 21.1667\\), so the last multiple of $6$ within the range is \\(21 \\times 6 = 126\\).\n3. List the multiples of $6$ between $6$ and $126$ inclusive: $6$, $12$, $18$, $\\dots$, $126$.\n4. The sequence of multiples forms an arithmetic sequence where each term increases by $6$. To find the number of terms, use the formula for the nth term of an arithmetic sequence: \\(a + (n-1)d\\), where \\(a\\) is the first term, \\(d\\) is the common difference, and \\(n\\) is the number of terms. Setting \\(6 + (n-1)6 = 126\\), we solve for \\(n\\):\n   \\[\n   6 + 6n - 6 = 126 \\implies 6n = 126 \\implies n = 21\n   \\]\n   This means there are 21 multiples of 6 between 5 and 127.\n\nConclusion: The number of multiples of 6 between 5 and 127 is $\\boxed{21}$.',
 'translated_solution': '\n1. Determ

In [146]:
df_judge = pd.DataFrame(l_judge_data)

for col in ['num_reasoning_steps', 'reasoning_step_content', 'same_math_expressions', 'overall_reasoning']:
    print(f"----- {col} -----")
    print(df_judge[col].value_counts())

----- num_reasoning_steps -----
num_reasoning_steps
Yes    496
No       4
Name: count, dtype: int64
----- reasoning_step_content -----
reasoning_step_content
Yes    327
No     173
Name: count, dtype: int64
----- same_math_expressions -----
same_math_expressions
No     497
Yes      3
Name: count, dtype: int64
----- overall_reasoning -----
overall_reasoning
Yes    413
No      87
Name: count, dtype: int64


In [151]:
sample = df_judge.sample(n=5).iloc[0]

print("REF -----")
print(sample['english_solution'])
print("TRANSLATED -----")
print(sample['translated_solution'])

REF -----
As given, $\overrightarrow {a}$ is perpendicular to $\overrightarrow {b}$, we have their dot product equal to zero:
$$\overrightarrow{a} \cdot \overrightarrow{b} = 0.$$ 
This means:
$$(\tan(\theta+ \frac{\pi}{12}), 1) \cdot (1, -2) = 0,$$
which simplifies to:
$$\tan(\theta+ \frac{\pi}{12}) \times 1 + 1 \times (-2) = 0.$$

Solving for $\tan(\theta+ \frac{\pi}{12})$, we get:
$$\tan(\theta+ \frac{\pi}{12}) = 2.$$

Now, let's find $\tan(2\theta+ \frac{\pi}{6})$ using the double angle identity for tangent:
$$\tan(2\theta+ \frac{\pi}{6}) = \frac {2 \tan(\theta+ \frac{\pi}{12})}{1 - \tan^2(\theta + \frac{\pi}{12})} = \frac{2 \times 2}{1 - 2^2} = -\frac{4}{3}.$$

Next, to find $\tan(2\theta+ \frac{5\pi}{12})$, we use the sum of angles formula for tangent, where the sum is $\frac{\pi}{6} + \frac{\pi}{4}$:
$$\tan(2\theta+ \frac{5\pi}{12}) = \tan(2\theta+ \frac{\pi}{6} + \frac{\pi}{4}) = \frac{\tan(2\theta+ \frac{\pi}{6}) + \tan(\frac{\pi}{4})}{1 - \tan(2\theta + \frac{\pi}{6})\tan(\fra